In [1]:
# # List all directories and files in the current working directory
# for root, dirs, files in os.walk('.'):
#     print("Root directory:", root)
#     print("Subdirectories:", dirs)
#     print("Files:", files)
#     break  # Stop after the first level to avoid printing too much

In [1]:
import os
os.chdir("../")    #commment after first run


In [42]:
#imports
import dask.distributed
import pystac_client
import planetary_computer
import stackstac 
import numpy as np
import pandas as pd
import rioxarray
import xarray as xr
import geopandas as gpd
from src.utils import gen_chips


In [43]:
print(os.getcwd())
#should be /home/benchuser/code

/home/benchuser/code


In [44]:
#os.chdir("/home/benchuser/code")

In [45]:
#config setup
import yaml
with open("notebooks/config.yml", "r") as file:
    config = yaml.safe_load(file)
print(config)  # Check the structure of the dictionary


{'sentinel_2': {'collection': 'sentinel-2-l2a', 'time_ranges': ['2023-01-01/2023-03-31', '2023-04-01/2023-06-30', '2023-07-01/2023-09-30', '2023-10-01/2023-12-31'], 'cloud_cover': 1, 'bands': ['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], 'resolution': 10}, 'land_cover': {'collection': 'io-lulc-annual-v02', 'year': '2023-01-01/2023-12-31'}, 'chips': {'sample_size': 256, 'chip_size': 128}, 'output': {'directory': 'notebooks/test_output_dump', 'naming_convention': 's2_{season}_{index:05}.tif'}, 'metadata': {'file': 'metadata.csv'}}


In [46]:
# Sentinel-2 settings
s2_collection = config["sentinel_2"]["collection"]
s2_date_ranges = config["sentinel_2"]["time_ranges"]
s2_bands = config["sentinel_2"]["bands"]
s2_resolution = config["sentinel_2"]["resolution"]
cloud_cover_threshold = config["sentinel_2"]["cloud_cover"]  # Max allowed cloud cover

# Land Cover settings
lc_collection = config["land_cover"]["collection"]
lc_year = config["land_cover"]["year"]  # Year of LC dataset

# Chip settings
sample_size = config["chips"]["sample_size"]  # Grid size for homogeneity check
chip_size = config["chips"]["chip_size"]  # Output chip size

# Output settings
output_dir = config["output"]["directory"]
chip_naming_convention = config["output"]["naming_convention"]

# Metadata settings
metadata_file = config["metadata"]["file"]

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define seasons for indexing
seasons = ["JFM", "AMJ", "JAS", "OND"]

aoi_gdf = gpd.read_file("data/urbans.geojson") # or "data/aois.geojson"


In [47]:
# # Get the first AOI using .iterrows()
# for _, aoi in aoi_gdf.iterrows():
#     aoi_bounds = aoi.geometry.bounds
#     break  # Exit after the first row
# # Extract coordinates
# minx, miny, maxx, maxy = aoi_bounds

# # Reduce size by 50% (adjustable)
# shrink_factor = 0.5  
# center_x = (minx + maxx) / 2
# center_y = (miny + maxy) / 2

# # Compute new bounds
# new_minx = center_x - (maxx - minx) * shrink_factor / 2
# new_maxx = center_x + (maxx - minx) * shrink_factor / 2
# new_miny = center_y - (maxy - miny) * shrink_factor / 2
# new_maxy = center_y + (maxy - miny) * shrink_factor / 2

# # Create new AOI dictionary (to pass into bbox-based queries)
# aoi_test_bbox = [new_minx, new_miny, new_maxx, new_maxy]

# print("Old AOI Bounding Box:", aoi_bounds)

# print("New AOI Bounding Box for Testing:", aoi_test_bbox)

In [48]:
#dask
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(cluster)
print(client.dashboard_link)

/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40191 instead
  warnings.warn(


http://127.0.0.1:40191/status


In [59]:

def search_s2_scenes(aoi, date_range):
    print(f"Searching for Sentinel-2 scenes for AOI at {aoi_test_bbox} within {date_range}")
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )
    s2_search = catalog.search(
        collections=["sentinel-2-l2a"],
        #bbox=aoi.geometry.bounds, #UNCOMMENT FOR FULL AOI
        bbox = aoi_test_bbox,
        datetime=date_range,
        query=["eo:cloud_cover<1"],
        sortby=["+properties.eo:cloud_cover"],
        max_items=1,
    )
    items = s2_search.item_collection()
    print(f"Found {len(items)} Sentinel-2 scenes")
    return items

def search_lc_scene(bbox, lc_date_range):
    print(f"Searching for Land Cover scenes within {lc_date_range} for bbox {bbox}")
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )
    lc_search = catalog.search(
        collections=["io-lulc-annual-v02"],
        bbox=bbox,
        datetime=lc_date_range,
    )
    items = lc_search.item_collection()
    print(f"Found {len(items)} Land Cover scenes")
    return items

def stack_s2_data(s2_items, s2_assets):
    if not s2_items:
        print("No Sentinel-2 items found.")
        return None
    
    try:
        print("Stacking Sentinel-2 images...")
        stacked_data = stackstac.stack(
            s2_items,
            assets=s2_assets,
            epsg=s2_items[0].properties["proj:epsg"],
            resolution=10,
            bounds_latlon=s2_items[0].bbox,
        ).median("time", skipna=True).squeeze()

        print("Stacked S2 data shape:", stacked_data.shape)

        if stacked_data is None or not isinstance(stacked_data, xr.DataArray):
            print("Error: Stacking Sentinel-2 data resulted in an invalid output.")
            return None

        return stacked_data

    except Exception as e:
        print(f"Error stacking Sentinel-2 data: {e}")
        return None

def stack_lc_data(lc_items, s2_epsg):
    if len(lc_items) == 0:
        print("No Land Cover data found.")
        return None
    try:
        print("Stacking Land Cover images...")

        stacked_data = stackstac.stack(
            lc_items,
            dtype=np.ubyte,
            fill_value=255,
            sortby_date=False,
            epsg=s2_epsg,
            resolution=10,
            bounds_latlon=lc_items[0].bbox
        ).squeeze()

        #Apply chunking after stacking
        stacked_data = stacked_data.chunk({"x": 1024, "y": 1024})

        print("Stacked LC data shape:", stacked_data.shape)
        print(f"Chunk sizes: {stacked_data.chunks}")

        return stacked_data

    except Exception as e:
        print(f"Error stacking Land Cover data: {e}")
        return None


def unique_class(window, axis=None, **kwargs):
    return np.all(window == window[:, :1, :, :1], axis=(1, 3))


def is_homogeneous(lc_stack):
    print("Checking for homogeneous LC regions...")

    # Ensure dimensions are exactly divisible by `sample_size`
    height, width = lc_stack.shape[-2], lc_stack.shape[-1]
    new_height = (height // sample_size) * sample_size
    new_width = (width // sample_size) * sample_size

    if new_height == 0 or new_width == 0:
        print("Error: LC stack is empty after cropping. Skipping this AOI.")
        return None  

    if new_height != height or new_width != width:
        print(f"Cropping LC stack to ({new_height}, {new_width}) to match sample_size.")
        lc_stack = lc_stack.isel(x=slice(0, new_height), y=slice(0, new_width))

    # Drop mismatched coordinate variables
    lc_stack = lc_stack.drop_vars([var for var in lc_stack.coords if "x" in var or "y" in var], errors="ignore")

    # Ensure rechunking aligns with the coarsening window
    lc_stack = lc_stack.chunk({"x": sample_size, "y": sample_size})

    # Check if lc_stack is empty after all modifications
    if lc_stack.size == 0:
        print("Error: LC stack is empty after cropping. Skipping this AOI.")
        return None  

    # Apply coarsening and reduce with unique_class function
    lc_uniqueness = lc_stack.coarsen(x=sample_size, y=sample_size, boundary="trim").reduce(unique_class)
    print("shape of lc_uniqueness:", lc_uniqueness.shape)
    print("datatype of lc_uniqueness:", lc_uniqueness.dtype)
    print("coordinates of lc_uniqueness:", list(lc_uniqueness.coords))
    print("type of lc_uniqueness:", type(lc_uniqueness))
    # Debugging: Print dtype of lc_uniqueness
    print(f"LC uniqueness dtype: {lc_uniqueness.dtype}, shape: {lc_uniqueness.shape}, type: {type(lc_uniqueness)}")

    return lc_uniqueness

    
def has_missing_values(array):
    print("Checking for missing values...")
    return array.isnull().any().compute()

def process_chips(aoi, s2_stack, lc_stack, output_dir, global_index):
    print(f"Processing chips for AOI at {aoi_test_bbox}")

    # Compute lc_uniqueness before looping
    lc_uniqueness = is_homogeneous(lc_stack).compute()
    if lc_uniqueness is None or lc_uniqueness.size == 0:
        print("skipping this aoi, lc_uniqueness is size 0 or None")
        return global_index
    print("shape of lc_uniqueness:", lc_uniqueness.shape)
    print("datatype of lc_uniqueness:", lc_uniqueness.dtype)
    print("coordinates of lc_uniqueness:", list(lc_uniqueness.coords))
    print("type of lc_uniqueness:", type(lc_uniqueness))
    
    print(f"LC uniqueness dtype after compute: {lc_uniqueness.dtype}, shape: {lc_uniqueness.shape}")

    for i in range(0, lc_stack.shape[1] - chip_size, sample_size):
        for j in range(0, lc_stack.shape[2] - chip_size, sample_size):
            
            # Ensure indices stay within valid bounds
            x_index = min(i // sample_size, lc_uniqueness.sizes["x"] - 1)
            y_index = min(j // sample_size, lc_uniqueness.sizes["y"] - 1)

            # Extract boolean value safely
            value = lc_uniqueness.isel(x=x_index, y=y_index).values
            print(f"checking lc_uniqueness at ({x_index}, {y_index}): {value}, Type: {type(value)}, Shape: {value.shape}")
            if isinstance(value, np.ndarray) and value.size == 1:
                value = value.item()

            
            if isinstance(value, np.ndarray):  
                value = value.item()  # Ensure it's a single boolean value

            if not value:
                print(f"Skipping chip at ({i}, {j}): LC region not homogeneous.")
                continue  

            s2_chip = s2_stack.isel(x=slice(i, i + chip_size), y=slice(j, j + chip_size))
            lc_chip = lc_stack.isel(x=slice(i, i + chip_size), y=slice(j, j + chip_size))

            if has_missing_values(s2_chip):
                continue  

            chip_name = f"s2_{date_range[:3]}_{global_index:05}.tif"
            lc_chip.rio.to_raster(os.path.join(output_dir, f"lc_{chip_name}"))
            s2_chip.rio.to_raster(os.path.join(output_dir, chip_name))

            print(f"Saved chip {chip_name}")
            global_index += 1

    return global_index

# def crop_to_aoi(stack, bbox):
#     """Crop the Sentinel-2 stack to match the AOI bounding box"""
#     minx, miny, maxx, maxy = bbox
#     return stack.rio.clip_box(minx, miny, maxx, maxy)



In [60]:
## Main processing loop
chip_dict = {}
global_index = 0
# Main processing loop
for index, aoi in aoi_gdf.iterrows():
    # Define the reduced AOI inside the loop
    aoi_bounds = aoi.geometry.bounds  # Full AOI bounds
    minx, miny, maxx, maxy = aoi_bounds

    # Reduce size by a factor (adjustable)
    shrink_factor = 0.3  # Reduce AOI size to 30%
    
    center_x = (minx + maxx) / 2
    center_y = (miny + maxy) / 2

    new_minx = center_x - (maxx - minx) * shrink_factor / 2
    new_maxx = center_x + (maxx - minx) * shrink_factor / 2
    new_miny = center_y - (maxy - miny) * shrink_factor / 2
    new_maxy = center_y + (maxy - miny) * shrink_factor / 2

    # Use the reduced AOI
    aoi_test_bbox = [new_minx, new_miny, new_maxx, new_maxy]
    
    print(f"Original AOI bounds: {aoi_bounds}")
    print(f"Reduced AOI bounds: {aoi_test_bbox}")
    print(f"Processing AOI {index} at {aoi_test_bbox}")

    # Search for Sentinel-2 images
    s2_items = [search_s2_scenes(aoi, date_range) for date_range in s2_date_ranges]
    
    if any(len(items) == 0 for items in s2_items):
        print(f"Skipping AOI {index}: Missing Sentinel-2 data for at least one quarter.")
        continue

    # Stack Sentinel-2 images
    s2_stack = stack_s2_data([item[0] for item in s2_items if item], s2_bands)

    # Ensure valid Sentinel-2 stack before proceeding
    if s2_stack is None:
        print(f"Skipping AOI {index}: No valid Sentinel-2 data found after cropping.")
        continue

    if not hasattr(s2_stack, "rio") or s2_stack.rio.crs is None:
        print(f"Error: s2_stack is missing CRS information. Assigning manually.")
        s2_first_item = next((item[0] for item in s2_items if item), None)
        if s2_first_item is None:
            print("Error: No valid Sentinel-2 item found.")
            continue

        s2_epsg = s2_first_item.properties["proj:epsg"]
        s2_stack.rio.write_crs(f"epsg:{s2_epsg}", inplace=True)

    print(f"Sentinel-2 CRS: {s2_stack.rio.crs}")

    # Search for Land Cover images
    lc_items = search_lc_scene(s2_items[0][0].bbox, lc_year)
    
    if len(lc_items) == 0:
        print(f"Skipping AOI {index}: No valid LC data found.")
        continue

    # Stack and process LC data
    print(f"LC BBOX: {lc_items[0].bbox}")  # Debug LC item bbox
    print(f"AOI BBOX: {aoi_test_bbox}")    # Debug AOI bbox
    lc_stack = stack_lc_data(lc_items, s2_stack.rio.crs.to_epsg())

    if lc_stack is None:
        print(f"Skipping AOI {index}: No valid LC data found.")
        continue

    # Process chips
    global_index, chip_dict = process_chips(aoi, s2_stack, lc_stack, output_dir, global_index)


    if global_index > 5:
        break




Original AOI bounds: (31.20416267326229, 30.02337142235983, 31.280433633102717, 30.060612342021983)
Reduced AOI bounds: [31.230857509206437, 30.036405744241584, 31.25373879715857, 30.04757802014023]
Processing AOI 0 at [31.230857509206437, 30.036405744241584, 31.25373879715857, 30.04757802014023]
Searching for Sentinel-2 scenes for AOI at [31.230857509206437, 30.036405744241584, 31.25373879715857, 30.04757802014023] within 2023-01-01/2023-03-31
Found 1 Sentinel-2 scenes
Searching for Sentinel-2 scenes for AOI at [31.230857509206437, 30.036405744241584, 31.25373879715857, 30.04757802014023] within 2023-04-01/2023-06-30
Found 1 Sentinel-2 scenes
Searching for Sentinel-2 scenes for AOI at [31.230857509206437, 30.036405744241584, 31.25373879715857, 30.04757802014023] within 2023-07-01/2023-09-30
Found 1 Sentinel-2 scenes
Searching for Sentinel-2 scenes for AOI at [31.230857509206437, 30.036405744241584, 31.25373879715857, 30.04757802014023] within 2023-10-01/2023-12-31
Found 1 Sentinel-2 s

/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


Found 2 Land Cover scenes
LC BBOX: [29.769589739624692, 23.97069843799003, 36.23041026037532, 32.035310832136304]
AOI BBOX: [31.230857509206437, 30.036405744241584, 31.25373879715857, 30.04757802014023]
Stacking Land Cover images...
Stacked LC data shape: (2, 89417, 65754)
Chunk sizes: ((1, 1), (1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 329), (1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 

/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


shape of lc_uniqueness: (2, 256, 256)
datatype of lc_uniqueness: bool
coordinates of lc_uniqueness: ['time', 'id', 'band', 'start_datetime', 'io:tile_id', 'end_datetime', 'proj:epsg', 'supercell', 'proj:shape', 'proj:transform', 'io:supercell_id', 'raster:bands', 'epsg']
type of lc_uniqueness: <class 'xarray.core.dataarray.DataArray'>
LC uniqueness dtype: bool, shape: (2, 256, 256), type: <class 'xarray.core.dataarray.DataArray'>


/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 37.46 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


shape of lc_uniqueness: (2, 256, 256)
datatype of lc_uniqueness: bool
coordinates of lc_uniqueness: ['time', 'id', 'band', 'start_datetime', 'io:tile_id', 'end_datetime', 'proj:epsg', 'supercell', 'proj:shape', 'proj:transform', 'io:supercell_id', 'raster:bands', 'epsg']
type of lc_uniqueness: <class 'xarray.core.dataarray.DataArray'>
LC uniqueness dtype after compute: bool, shape: (2, 256, 256)
checking lc_uniqueness at (0, 0): [False False], Type: <class 'numpy.ndarray'>, Shape: (2,)


ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
#optional plotting code
import matplotlib.pyplot as plt

# Select the first stored location
if chip_dict:
    first_location = next(iter(chip_dict.keys()))  # Get the first (i, j) key
    first_four_chips = chip_dict[first_location]  # Get the corresponding chips

    fig, axes = plt.subplots(2, 4, figsize=(15, 6))  # 2 rows, 4 columns

    for idx, (s2_chip, lc_chip) in enumerate(first_four_chips):
        # Sentinel-2 RGB Visualization (assuming B4, B3, B2 are RGB)
        rgb_image = s2_chip.sel(band=[3, 2, 1]).transpose("y", "x", "band")  # Adjust band indices if needed
        axes[0, idx].imshow(rgb_image.compute().clip(0, 3000) / 3000)  # Normalize for better visualization
        axes[0, idx].set_title(f"Sentinel-2 Chip {idx+1}")
        axes[0, idx].axis("off")

        # Land Cover Visualization
        axes[1, idx].imshow(lc_chip.compute(), cmap="tab10")  # Color map for categorical data
        axes[1, idx].set_title(f"Land Cover Chip {idx+1}")
        axes[1, idx].axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
print(chip_dict)